# Informer Demo

## Download code and dataset

In [ ]:
import sys
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

In [ ]:
# !pip install -r ./Informer2020/requirements.txt

In [3]:
import pandas as pd
data = pd.read_csv("./dataset/ETDataset/ETTm1.csv")
data.date = pd.to_datetime(data.date)
data = data.set_index('date')

In [5]:
data

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
date,,,,,,,
2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
2016-07-01 00:15:00,5.760,2.076,1.492,0.426,4.264,1.401,30.459999
2016-07-01 00:30:00,5.760,1.942,1.492,0.391,4.234,1.310,30.038000
2016-07-01 00:45:00,5.760,1.942,1.492,0.426,4.234,1.310,27.013000
2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
...,...,...,...,...,...,...,...
2018-06-26 18:45:00,9.310,3.550,5.437,1.670,3.868,1.462,9.567000
2018-06-26 19:00:00,10.114,3.550,6.183,1.564,3.716,1.462,9.567000
2018-06-26 19:15:00,10.784,3.349,7.000,1.635,3.746,1.432,9.426000


In [6]:
train_df = data.iloc[:10*30*24*4]

In [7]:
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# scaler = MinMaxScaler()
scaler = StandardScaler()
scaler.fit(train_df)
trans_df = scaler.transform(data)
norm_data = pd.DataFrame(data=trans_df, index=data.index, columns=data.columns, dtype=None, copy=None)

In [11]:
norm_data.describe()

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
count,69680.000000,69680.000000,69680.000000,69680.000000,69680.000000,69680.000000,69680.000000
mean,-0.294849,0.101654,-0.358719,0.107309,0.225105,0.042873,-0.372737
std,1.383776,0.917197,1.449403,0.886133,1.069708,1.002302,0.861735
min,-6.284042,-3.472437,-6.872093,-3.294317,-3.666171,-3.677795,-2.137661
25%,-0.604896,-0.553195,-0.574611,-0.471523,-0.474530,-0.270797,-1.012301
50%,-0.016057,0.078551,-0.001573,0.137950,-0.002572,0.238333,-0.566382
75%,0.559692,0.740851,0.556609,0.747913,0.746366,0.643968,0.106017
max,2.980689,3.720301,2.562669,3.518243,5.158902,3.695409,2.915948


In [13]:
import joblib
joblib.dump(scaler, './dataset/ETDataset/ETT_standard_scaler.pkl')

['./dataset/ETDataset/ETT_standard_scaler.pkl']

## Experiments: Train and Test

In [1]:
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch

In [2]:
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'ETTm1' # data
args.root_path = './dataset/ETDataset/' # root path of data file
args.data_path = 'ETTm1.csv' # data file
args.features = 'M' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'OT' # target feature in S or MS task
args.freq = 't' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

# 4*12=48, 4*12*2=96, 4*12*4=192, 4*12*8=384, 4*12*16=768
args.seq_len = 4*12 # input sequence length of Informer encoder
args.label_len = args.seq_len//2 # start token length of Informer decoder
args.pred_len = 48 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 7 # encoder input size
args.dec_in = 7 # decoder input size
args.c_out = 7 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0

args.batch_size = 32 
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 8
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'


In [ ]:
# 원본
import time
start = time.time()  # 시작 시간 저장start = time.time()  # 시작 시간 저장

for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

    # set experiments
    exp = Exp(args)
    
    # train
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)
    
    # test
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting)

    torch.cuda.empty_cache()

f = open('./results/'+setting+ "/time.txt", 'w')
f.write(str(time.time() - start))
f.close()

In [ ]:
################################################################################

In [ ]:
# 위에서는 validation 함수 호출을 안했었음
# learning rate이랑 s_layer, d_layer 바꿈
# exp_information에서 train과 validatioin 함수에서 _get_data 함수의 flag를 'val'에서 'validation'으로 바꿈 --> 비슷해서 다시 원상복구('val'로 바꿔놓음)
# 각 case 별 메모리가 언제 터지는지 확인

# ProbSparse를 사용해도 빨라지지 않는 이유 -->  https://github.com/zhouhaoyi/Informer2020/issues/142

In [1]:
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch
import time

In [2]:
# 속도도 일단은 그냥 하는게 좋을듯. 어쨋든 distilling은 시간이 줄어드니까. probsparse는 속도안줄어든다고 적어놓고
# 결과가 안좋았던 이유가 계속 label_len이 24로 고정임 --> 그래도 안좋네
# lr type 2로

In [ ]:
# informer랑 informer stack으로

In [2]:
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch

args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'ETTm1' # data
args.root_path = './dataset/ETDataset/' # root path of data file
args.data_path = 'ETTm1.csv' # data file
args.features = 'M' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'OT' # target feature in S or MS task
args.freq = 't' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

# 4*12=48, 4*12*2=96, 4*12*4=192, 4*12*8=384, 4*12*16=768
args.seq_len = 4*12 # input sequence length of Informer encoder
args.label_len = args.seq_len//2 # start token length of Informer decoder
args.pred_len = 48 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 7 # encoder input size
args.dec_in = 7 # decoder input size
args.c_out = 7 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads

args.e_layers = 4 # num of encoder layers
#args.s_layers = [3, 2, 1]
args.d_layers = 2 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0

args.batch_size = 32 
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type2'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 10
args.patience = 5
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'

In [3]:
# 4*12*24 까지는 메모리 안터짐
#seq_len_list = [4*12, 4*12*2, 4*12*4, 4*12*8, 4*12*16, 4*12*24]
seq_len_list = [4*12*4]
args.attn = 'prob'
args.distil = True

for seq_len in seq_len_list:
    args.seq_len = seq_len
    args.label_len = args.seq_len//2
    Exp = Exp_Informer

    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                    args.seq_len, args.label_len, args.pred_len,
                    args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

        # set experiments
        exp = Exp(args)

        # train
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        # validation
        print('>>>>>>>validation : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.validation(setting)
        # test
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()
        
    start = time.time()
    exp.test(setting)
    f = open('./results/'+setting+ "/time.txt", 'w')
    f.write(str(time.time() - start))
    f.close()

Use GPU: cuda:0
>>>>>>>start training : informer_ETTm1_ftM_sl192_ll96_pl48_dm512_nh8_el4_dl2_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 34321
val 11473
test 11473
	iters: 100, epoch: 1 | loss: 0.4589823
	speed: 0.0810s/iter; left time: 860.7575s
	iters: 200, epoch: 1 | loss: 0.2585064
	speed: 0.0761s/iter; left time: 800.7519s
	iters: 300, epoch: 1 | loss: 0.2602194
	speed: 0.0763s/iter; left time: 794.8999s
	iters: 400, epoch: 1 | loss: 0.2538326
	speed: 0.0764s/iter; left time: 788.9585s
	iters: 500, epoch: 1 | loss: 0.2383778
	speed: 0.0776s/iter; left time: 793.5131s
	iters: 600, epoch: 1 | loss: 0.1705451
	speed: 0.0768s/iter; left time: 777.0149s
	iters: 700, epoch: 1 | loss: 0.2315241
	speed: 0.0769s/iter; left time: 770.1181s
	iters: 800, epoch: 1 | loss: 0.1887291
	speed: 0.0767s/iter; left time: 760.7995s
	iters: 900, epoch: 1 | loss: 0.1729531
	speed: 0.0810s/iter; left time: 795.7501s
	iters: 1000, epoch: 1 | loss: 0.1727644
	speed: 0.0802

In [3]:
# 4*12*16 까지는 메모리 안터짐
seq_len_list = [4*12, 4*12*2, 4*12*4, 4*12*8, 4*12*16]
args.attn = 'full'
args.distil = True

for seq_len in seq_len_list:
    args.seq_len = seq_len
    args.label_len = args.seq_len//2
    
    Exp = Exp_Informer

    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                    args.seq_len, args.label_len, args.pred_len,
                    args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

        # set experiments
        exp = Exp(args)

        # train
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        # validation
        print('>>>>>>>validation : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.validation(setting)
        # test
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()
        
    start = time.time()
    exp.test(setting)
    f = open('./results/'+setting+ "/time.txt", 'w')
    f.write(str(time.time() - start))
    f.close()

Use GPU: cuda:0
>>>>>>>start training : informerstack_ETTm1_ftM_sl48_ll24_pl48_dm512_nh8_elNone_dl2_df2048_atfull_fc5_ebtimeF_dtTrue_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 28705
val 11473
test 5713
	iters: 100, epoch: 1 | loss: 0.3625029
	speed: 0.0443s/iter; left time: 392.9335s
	iters: 200, epoch: 1 | loss: 0.2542932
	speed: 0.0401s/iter; left time: 352.0168s
	iters: 300, epoch: 1 | loss: 0.2421058
	speed: 0.0394s/iter; left time: 341.7303s
	iters: 400, epoch: 1 | loss: 0.2103720
	speed: 0.0392s/iter; left time: 335.6565s
	iters: 500, epoch: 1 | loss: 0.1903096
	speed: 0.0394s/iter; left time: 333.7224s
	iters: 600, epoch: 1 | loss: 0.1997913
	speed: 0.0394s/iter; left time: 329.4581s
	iters: 700, epoch: 1 | loss: 0.2010699
	speed: 0.0390s/iter; left time: 322.3992s
	iters: 800, epoch: 1 | loss: 0.2286577
	speed: 0.0393s/iter; left time: 320.7291s
Epoch: 1 cost time: 35.79236388206482
Epoch: 1, Steps: 897 | Train Loss: 0.2455541 Vali Loss: 0.8114886 Test Loss: 0.4691055
Validat

In [4]:
# 4*12*16 까지는 메모리 안터짐
seq_len_list = [4*12, 4*12*2, 4*12*4, 4*12*8, 4*12*16]
args.attn = 'prob'
args.distil = False

for seq_len in seq_len_list:
    args.seq_len = seq_len
    args.label_len = args.seq_len//2
    
    Exp = Exp_Informer

    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                    args.seq_len, args.label_len, args.pred_len,
                    args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

        # set experiments
        exp = Exp(args)

        # train
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        # validation
        print('>>>>>>>validation : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.validation(setting)
        # test
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()
        
    start = time.time()
    exp.test(setting)
    f = open('./results/'+setting+ "/time.txt", 'w')
    f.write(str(time.time() - start))
    f.close()

Use GPU: cuda:0
>>>>>>>start training : informerstack_ETTm1_ftM_sl48_ll24_pl48_dm512_nh8_elNone_dl2_df2048_atprob_fc5_ebtimeF_dtFalse_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 28705
val 11473
test 5713
	iters: 100, epoch: 1 | loss: 0.3182735
	speed: 0.0536s/iter; left time: 475.5303s
	iters: 200, epoch: 1 | loss: 0.2353503
	speed: 0.0538s/iter; left time: 471.9450s
	iters: 300, epoch: 1 | loss: 0.2513919
	speed: 0.0539s/iter; left time: 467.1990s
	iters: 400, epoch: 1 | loss: 0.2594191
	speed: 0.0538s/iter; left time: 461.4334s
	iters: 500, epoch: 1 | loss: 0.2291717
	speed: 0.0540s/iter; left time: 457.6102s
	iters: 600, epoch: 1 | loss: 0.2159271
	speed: 0.0539s/iter; left time: 450.8291s
	iters: 700, epoch: 1 | loss: 0.1787126
	speed: 0.0541s/iter; left time: 447.3148s
	iters: 800, epoch: 1 | loss: 0.1834159
	speed: 0.0539s/iter; left time: 440.7814s
Epoch: 1 cost time: 48.37083029747009
Epoch: 1, Steps: 897 | Train Loss: 0.2566530 Vali Loss: 0.7703239 Test Loss: 0.4393870
Valida

KeyboardInterrupt: 

In [ ]:
# 4*12*8 까지는 메모리 안터짐
seq_len_list = [4*12, 4*12*2, 4*12*4, 4*12*8]
args.attn = 'full'
args.distil = False


for seq_len in seq_len_list:
    args.seq_len = seq_len
    args.label_len = args.seq_len//2
    
    Exp = Exp_Informer

    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                    args.seq_len, args.label_len, args.pred_len,
                    args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

        # set experiments
        exp = Exp(args)

        # train
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        # validation
        print('>>>>>>>validation : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.validation(setting)
        # test
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)

        torch.cuda.empty_cache()
        
    start = time.time()
    exp.test(setting)
    f = open('./results/'+setting+ "/time.txt", 'w')
    f.write(str(time.time() - start))
    f.close()